<a href="https://colab.research.google.com/github/silasabs/CoherentOptics/blob/main/examples/7.%20Clock%20Recovery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
if 'google.colab' in str(get_ipython()):    
    ! git clone -b main https://github.com/edsonportosilva/OptiCommPy
    from os import chdir as cd
    cd('/content/OptiCommPy/')
    ! pip install . 

In [3]:
import matplotlib.pyplot as plt
import numpy as np

from optic.dsp.core import pulseShape, firFilter, decimate, pnorm
from optic.models.devices import pdmCoherentReceiver, basicLaserModel
from optic.models.channels import linearFiberChannel, awgn
from optic.models.tx import simpleWDMTx
from optic.utils import parameters
from optic.comm.metrics import fastBERcalc
from optic.comm.modulation import grayMapping
from optic.plot import pconst

import scipy.constants as const

# Clock Recovery

Os sistemas de comunicação digital processam amostras digitalizadas de formas de onda analógicas. Essas amostras devem ser coletadas em instantes específicos de tempo, e imperfeições nesse processo podem prejudicar significativamente a transmissão de dados. 

Erros de temporização podem ser classificados em erros de fase de amostragem e erros de frequência de amostragem. Erros de fase de amostragem aparecem quando a frequência do relógio do receptor é sincronizada com precisão com a frequência do relógio do transmissor, mas há um atraso constante com relação aos instantes de amostragem ideais. 

Em sistemas com erros de frequência de amostragem, os instantes de amostragem ideais variam ao longo do tempo, levando a uma resposta de frequência variável no tempo que não pode ser rastreada indefinidamente. Portanto, algoritmos de recuperação de clock, que corrigem adaptativamente erros de amostragem de frequência, são de fundamental importância.

## 7.1 Clock Recovery Architectures

Os esquemas de recuperação de relógio em sistemas digitais podem ser classificados de acordo com sua arquitetura em três categorias:

- **Esquemas analógicos:** Em um esquema puramente analógico, um circuito de entrada analógico extrai informações de tempo do sinal e atua diretamente. Tais estruturas de recuperação de relógio analógico não são usadas em receptores ópticos coerentes digitais, pois a detecção de erro de tempo pode ser implementada efetivamente no domínio digital.

<br>
<center>
    <img src="https://i.postimg.cc/4xHQqpC4/Screenshot-from-2024-10-04-18-21-31.png" width="900">
</center>
</br>

- **Esquemas Híbridos:** No esquema híbrido, o sinal recebido é amostrado e processado no domínio digital para extrair informações de tempo. Essas informações são então usadas para atuar no relógio de amostragem.

<br>
<center>
    <img src="https://i.postimg.cc/GmbsJYcS/Screenshot-from-2024-10-04-18-26-01.png" width="900">
</center>
</br>

- **Esquemas Digitais:** No esquema puramente digital, a extração do sinal de relógio, bem como os ajustes de tempo de amostragem são feitos de forma puramente digital.

<br>
<center>
    <img src="https://i.postimg.cc/N0SMVrct/Screenshot-from-2024-10-04-18-27-15.png" width="900">
</center>
</br>

Existem várias alternativas viáveis ​​para implementar a recuperação de relógio em sistemas ópticos coerentes digitais, incluindo esquemas híbridos analógico-digitais, esquemas puramente digitais ou uma combinação de ambos. Uma solução comum combina
um esquema híbrido analógico-digital para ajustes de tempo grosseiros e um esquema puramente digital para ajustes finos de tempo.

## Implementações nos domínios do tempo e frequência

Algoritmos de recuperação de clock podem ser implementados no domínio do tempo ou frequência. Algoritmos no domínio de frequência podem ser vantajosos do ponto de vista da complexidade se a cadeia de algoritmos DSP já implementa transformadas rápidas de Fourier (FFTs) e transformadas rápidas de Fourier inversas (IFFTs) para outras aplicações, como a compensação da dispersão cromática, evitando operações repetidas.

Um esquema híbrido típico de domínio de frequência implantado em sistemas de comunicações ópticas é mostrado abaixo. Um sinal de detector de erro de temporização (TED) é obtido no domínio de frequência após o sinal ser multiplicado pelo filtro de compensação CD, mas antes do IFFT.

<br>
<center>
    <img src="https://i.postimg.cc/DyTcHSv1/Screenshot-from-2024-10-04-18-44-55.png">
</center>
</br>

A saída do TED fornece um sinal de controle que, após a filtragem no filtro de loop (LF), é capaz de controlar o oscilador controlado por tensão (VCO) que atua nos ADCs. Um esquema puramente digital típico de domínio de tempo é mostrado abaixo.

O sinal de entrada $x(t)$ é primeiramente amostrado pelo ADC a uma taxa de amostragem fixa $1/Ta$ , onde $Ta$ é o período de amostragem. Assim, um sinal digitalizado de tempo discreto, com amostras $x[m] = x(mTa)$, é produzido na saída do ADC. O sinal digital é então encaminhado para um interpolador, que amostra o sinal nos tempos de amostragem desejados.

<br>
<center>
    <img src="https://i.postimg.cc/mkG7vnsT/Screenshot-from-2024-10-04-18-53-12.png">
</center>
</br>

A saída do interpolador gera amostras com taxa $1/T_I$ . Idealmente, como sistemas ópticos coerentes digitais trabalham com equalizadores $Ts/2$ espaçados fracionadamente, $T_I = Ts/2$. O caminho de feedback começa com um TED, fornecendo um sinal de controle que, após o LF, é capaz de controlar um oscilador controlado numericamente (NCO). A saída do NCO atua no interpolador, fechando o loop de fase digital (DPLL). Várias outras combinações também são possíveis, por exemplo, com recuperação de clock puramente digital no domínio da frequência.

## 7.2 Interpolador

Em esquemas de recuperação de clock puramente digitais, o propósito da interpolação é obter uma amostra da forma de onda do sinal em um determinado instante, com base em amostras coletadas em instantes vizinhos. 

<br>
<center>
    <img src="https://i.postimg.cc/XqptPxwN/Screenshot-from-2024-10-04-20-44-12.png">
</center>
</br>

Um conjunto de amostras de entrada espaçadas em $T_a$, indicadas na parte superior da figura, são usadas para gerar a amostra de saída no instante $nT_I$ . O processo é baseado na interpolação de uma amostra localizada em um intervalo fracionário $\mu_n T_a$ , tendo como ponto base uma amostra no instante $m_n T_a$ , onde $m_n = \frac{kT_I}{T_a}$. 
A interpolação já foi discutida no [Cap. 4](https://github.com/silasabs/CoherentOptics/blob/main/examples/4.%20Coherent%20Receiver%20Front-End.ipynb), no escopo da compensação de inclinação. A recuperação de clock usa algoritmos semelhantes, que são baseados em filtros lineares transversais. No entanto, diferentemente da compensação de inclinação, o esquema de recuperação de clock ajusta os coeficientes do filtro dinamicamente, símbolo por símbolo, com base na saída de um NCO. 


A interpolação pode ser implementada no domínio da frequência junto com a compensação de CD, ou usando um interpolador de domínio de tempo. Uma solução popular no domínio do tempo usa um interpolador cúbico de Lagrange. Os interpoladores cúbicos podem ter de 4 a 10 taps com banda passante crescente. Os coeficientes de um interpolador cúbico de 4 taps, para um instante de interpolação $t = m_nT_a + \mu_n T_a$ , são fornecidos por [2].

$$w_{n-2}\;=\;-\,{\frac{1}{6}}\mu_{n}^{3}+\,{\frac{1}{6}}\mu_{n};$$

$$w_{n-1}\;=\;\frac{1}{2}\mu_{n}^{3}+\frac{1}{2}\mu_{n}^{2}-\mu_{n};$$

$$w_{n}\;=\;-\,\frac{1}{2}\mu_{n}^{3}-\mu_{n}^{2}+\frac{1}{2}\mu_{n}+1;$$

$$w_{n+1}\;=\;{\frac{1}{6}}\mu_{n}^{3}+{\frac{1}{2}}\mu_{n}^{2}+{\frac{1}{3}}\mu_{n}.$$

Atualizar os coeficientes do filtro para cada intervalo fracionário $\mu_n$ exigiria o cálculo e a transferência de quatro coeficientes. Alternativamente, a estrutura de interpolação proposta por Farrow em [3] requer a transferência de uma única variável  $\mu_n$ , ao custo de uma estrutura de filtro mais complexa.

In [2]:
def interpolator(x, mu):
    """
    Interpolador cúbico baseado na estrutura de Farrow

    Parameters
    ----------
    x : np.array
        matriz de 4 elementos para interpolação cúbica.
    
    mu : float
        parâmetro de interpolação.

    Returns
    -------
    y : float
        sinal interpolado.
    
    Referências
    -----------
        [1] Digital Coherent Optical Systems, Architecture and Algorithms
        
        [2] C. Farrow, A continuously variable digital delay element, in IEEE International Symposium on Circuits and Systems, vol. 3 (1988), pp. 2641–2645
    """
    
    return (
        x[0] * (-1/6 * mu**3 + 0 * mu**2 + 1/6 * mu + 0) +
        x[1] * (1/2 * mu**3 + 1/2 * mu**2 - mu + 0) +
        x[2] * (-1/2 * mu**3 - mu**2 + 1/2 * mu + 1) +
        x[3] * (1/6 * mu**3 + 1/2 * mu**2 + 1/3 * mu + 0)
    )

## 7.3 Timing Error Detector

O TED extrai informações de tempo e as encaminha, após filtragem, para um VCO ou NCO. Este processo é realizado em malha fechada, construindo o chamado loop de fase bloqueada (PLL). Talvez o esquema TED mais utilizado seja o proposto por Gardner em [4]. Tomando três amostras vizinhas espaçadas em $Ts/2$ do sinal recebido $x[2k − 2], x[2k − 1]$ e $x[2k]$, assumidas como reais por uma questão de simplicidade, o algoritmo de Gardner calcula o sinal de indicação de erro de tempo espaçado em $Ts$ como:

$$e[k]=x[n-1](x[n]-x[n-2]),\ \ n=2k.$$

<br>
<center>
    <img src="https://i.postimg.cc/D0cSq3cW/Screenshot-from-2024-10-05-21-05-12.png">
</center>
</br>

- Figura (a): $\Delta\tau = 0$ (Tempo de amostragem correto)
- Figura (b): $\Delta\tau > 0$ (Amostragem tardia)
- Figura (c): $\Delta\tau < 0$ (Amostragem precoce)

O princípio básico do Gardner TED é descrito acima para um sinal BPSK. Os três círculos laranjas indicam as amostras de entrada usadas pelo algoritmo. Três casos são indicados. Em cada um desses casos temos, respectivamente,

$$e[k]=0\cdot(1-(-1))=0;$$
$$e[k]=0.2\cdot(0.8-(-0.8))=0.32\gt 0;$$
$$e[k]=-0.2\cdot(0.8-(-0.8))=-0.32\lt 0.$$

Se o sinal de entrada for amostrado na fase correta, a amostra central coincide com as transições, e o sinal de erro vai idealmente para zero. Se a fase de amostragem for tardia, a amostra central tem um valor positivo, e o sinal de $e[k]$ é positivo. Se, no entanto, a fase de amostragem for muito precoce, a amostra central tem um valor negativo, e o sinal de $e[k]$ é negativo. Portanto, o valor absoluto de $e[k]$ dá uma indicação da magnitude da incompatibilidade da fase de amostragem, enquanto seu sinal indica se a fase de amostragem é precoce ou tardia. Embora à primeira vista o algoritmo de Gardner pareça corrigir apenas erros de amostragem de fase, sua operação contínua e os ajustes feitos ao longo do tempo fornecem uma correção substancial para erros de amostragem de frequência.

Em sistemas com modulação em fase e em quadratura, o TED pode combinar ambos os componentes como:

$$\begin{array}{c}{{e[k]=x^{I}[n-1]\left(x^{I}[n]-x^{I}[n-2]\right)+x^{Q}[n-1]\left(x^{Q}[n]-x^{Q}[n-2]\right)}}\end{array}$$

$$\begin{array}{c}{{=\Re\left\{x^{*}[n-1]\left(x[n]-x[n-2]\right)\right\}, n = 2k.}}\end{array}$$

Conforme discutido anteriormente, desvios no instante de amostragem correto $\Delta\tau$ causam um aumento na magnitude da saída $e[k]$, e seu sinal indica um tempo de amostragem tardio ou precoce. O TED de Gardner requer transições bem definidas entre símbolos adjacentes, mas sistemas coerentes digitais ópticos são fortemente afetados por efeitos dispersivos.

Algoritmos TED de domínio de tempo são geralmente aplicados após o filtro de compensação CD, resolvendo metade do problema. A questão então é se é mais apropriado implementá-lo antes ou depois do filtro de compensação PMD adaptativo borboleta. Implementá-lo antes da compensação PMD, preserva o equalizador de mudanças estocásticas devido a variações de clock. Por outro lado, alguns casos patológicos de PMD podem fazer o sinal de clock desaparecer, deteriorando o desempenho do sistema.

Uma possível solução para esse problema é desfazer a rotação de polarização realizada pelo canal combinando os sinais das orientações V e H, mas essa estratégia requer um loop de rastreamento para o ângulo de rotação. Além disso, os efeitos de canal relacionados à polarização podem ser mais complicados do que um único elemento de atraso de grupo diferencial (DGD). Embora o TED de Gardner tenha um desempenho satisfatório com pulsos NRZ e RZ, ele é fortemente prejudicado por pulsos Nyquist com pequenos fatores de roll-off. O algoritmo proposto em [5] implementa um TED Gardner (MG) modificado com base na potência
dos símbolos equalizados.

In [ ]:
def gardnerTED(x, pulseType):
    """
    Gardner TED Algorithm.

    Parameters
    ----------
    x : np.array
        Matriz de três amostras para o calculo do erro.

    pulseType : string
        Tipo de pulso utilizado.

    Returns
    -------
    ek: float
        magnitude do erro de tempo.
    
    Referências
    -----------
        [1] Digital Coherent Optical Systems, Architecture and Algorithms

        [2] F. Gardner, A BPSK/QPSK timing-error detector for sampled receivers. IEEE Trans. Commun. 34(5), 423–429 (1986)
    """
    if pulseType == "Nyquist":
        ek = np.abs(x[1]) ** 2 * (np.abs(x[0]) ** 2 - np.abs(x[2]) ** 2)
    else:
        ek = np.real(np.conj(x[1]) * (x[2] - x[0]))
    return ek

# Referências

[1] Digital Coherent Optical Systems, Architecture and Algorithms

[2] L. Erup, F.M. Gardner, R.A. Harris, Interpolation in digital modems II. Implementation and performance. IEEE Trans. Commun. 41(6), 998–1008 (1993)

[3] C. Farrow, A continuously variable digital delay element, in IEEE International Symposium on Circuits and Systems, vol. 3 (1988), pp. 2641–2645

[4] F. Gardner, A BPSK/QPSK timing-error detector for sampled receivers. IEEE Trans. Commun. 34(5), 423–429 (1986)

[5] N. Stojanovic, C. Xie, Y. Zhao, B. Mao, N. Gonzalez, J. Qi, N. Binh, Modified Gardner phase detector for Nyquist coherent optical transmission systems, in Proceedings of Optical Fiber Communication Conference and National Fiber Optic Engineers Conference (OFC/NFOEC)
(Optical Society of America, America, 2013), p. JTh2A.50. [http://www.osapublishing.org/abstract.cfm?URI=OFC-2013-JTh2A.50](http://www.osapublishing.org/abstract.cfm?URI=OFC-2013-JTh2A.50)